In [102]:
# Import dependencies and API key
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from config import gkey
gmaps.configure(gkey)

In [103]:
# Import previous data and create dataframe
weather_df = pd.read_csv("../weatherpy/output/weatherpy.csv", encoding="utf-8")
weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,0,mataura,NZ,-46.1927,168.8643,1619229959,99,89,48.99,1.01
1,1,bambous virieux,MU,-20.2567,57.4061,1619229988,40,89,78.80,11.50
2,2,yellowknife,CA,62.4560,-114.3525,1619229938,75,57,15.80,9.22
3,3,faya,SA,18.3851,42.4509,1619229986,0,51,62.60,1.90
4,4,ushuaia,AR,-54.8000,-68.3000,1619229937,75,93,42.80,14.97


In [104]:
# Set humidity max as weight, use latitude and longitude as locations
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()
location = weather_df[["Latitude", "Longitude"]]

# Configure gmaps
fig = gmaps.figure()

# Add heat layer to map
heat_layer = gmaps.heatmap_layer(location, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [107]:
# Narrow down cities to fit weather conditions
ideal_weather_df = weather_df.loc[(weather_df["Max Temperature"] > 70) & (weather_df["Max Temperature"] < 80) & (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] < 7) & (weather_df["Humidity"] < 80), :]

# Drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
del ideal_weather_df["Unnamed: 0"]

# Display df
ideal_weather_df

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,sahuaripa,MX,29.0500,-109.2333,1619230171,0,15,71.51,4.54
1,geraldton,AU,-28.7667,114.6000,1619229968,0,43,75.20,6.91
2,dhankuta,NP,26.9833,87.3333,1619230208,0,21,73.98,3.56
3,riyadh,SA,24.6877,46.7219,1619229938,0,16,78.80,5.75
4,mansa,IN,29.9833,75.3833,1619230241,0,51,71.47,5.57
5,hafizabad,PK,32.0679,73.6854,1619230242,0,42,70.12,2.84
6,morondava,MG,-20.2833,44.2833,1619230243,0,79,74.30,6.17
7,meybod,IR,32.2501,54.0166,1619230253,0,22,75.20,4.61
8,buraydah,SA,26.3260,43.9750,1619230214,0,25,73.40,3.44
9,garhi yasin,IN,28.4955,77.4243,1619230260,0,49,71.60,4.61


In [108]:
# Set parameters, search Google Places, store hotels 
hotels = []

for i in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[i]['Latitude']
    lng = ideal_weather_df.loc[i]['Longitude']

    paramaters = {"location": f"{lat},{lng}", "radius": 5000, "types" : "hotel", "key": gkey}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=paramaters)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
        
# Add a "Hotel Name" column to the DataFrame
ideal_weather_df["Hotel Name"] = hotels
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name
0,sahuaripa,MX,29.0500,-109.2333,1619230171,0,15,71.51,4.54,Sahuaripa
1,geraldton,AU,-28.7667,114.6000,1619229968,0,43,75.20,6.91,Geraldton
2,dhankuta,NP,26.9833,87.3333,1619230208,0,21,73.98,3.56,Dhankuta
3,riyadh,SA,24.6877,46.7219,1619229938,0,16,78.80,5.75,Riyadh
4,mansa,IN,29.9833,75.3833,1619230241,0,51,71.47,5.57,Mansa
5,hafizabad,PK,32.0679,73.6854,1619230242,0,42,70.12,2.84,Hafizabad
6,morondava,MG,-20.2833,44.2833,1619230243,0,79,74.30,6.17,Morondava
7,meybod,IR,32.2501,54.0166,1619230253,0,22,75.20,4.61,Meybod
8,buraydah,SA,26.3260,43.9750,1619230214,0,25,73.40,3.44,Buraydah
9,garhi yasin,IN,28.4955,77.4243,1619230260,0,49,71.60,4.61,Noida


In [109]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [110]:
# Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))